In [255]:
import json
import time
from collections import Counter, defaultdict
from datetime import datetime
from tqdm import tqdm
from websockets import ConnectionClosedError
from web3 import Web3

convert_unixtime = lambda ts: datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

In [115]:
INFURA_URL = "https://mainnet.infura.io/v3/87dde21eb7704f8c82f3b51aa58653d0"
INFURA_PROVIDER = "wss://mainnet.infura.io/ws/v3/87dde21eb7704f8c82f3b51aa58653d0"
ETH_USD_PRICE_CONTRACT_ADDR = '0xF79D6aFBb6dA890132F9D7c355e3015f15F3406F'
CHAINLINK_ABI = json.loads('[{"constant":false,"inputs":[{"name":"_requestId","type":"bytes32"},{"name":"_payment","type":"uint256"},{"name":"_expiration","type":"uint256"}],"name":"cancelRequest","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"authorizedRequesters","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"uint256"}],"name":"jobIds","outputs":[{"name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"latestAnswer","outputs":[{"name":"","type":"int256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"minimumResponses","outputs":[{"name":"","type":"uint128"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"uint256"}],"name":"oracles","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_recipient","type":"address"},{"name":"_amount","type":"uint256"}],"name":"transferLINK","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"latestRound","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_clRequestId","type":"bytes32"},{"name":"_response","type":"int256"}],"name":"chainlinkCallback","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"renounceOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_paymentAmount","type":"uint128"},{"name":"_minimumResponses","type":"uint128"},{"name":"_oracles","type":"address[]"},{"name":"_jobIds","type":"bytes32[]"}],"name":"updateRequestDetails","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"latestTimestamp","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"destroy","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_roundId","type":"uint256"}],"name":"getAnswer","outputs":[{"name":"","type":"int256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_roundId","type":"uint256"}],"name":"getTimestamp","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"paymentAmount","outputs":[{"name":"","type":"uint128"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"requestRateUpdate","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_requester","type":"address"},{"name":"_allowed","type":"bool"}],"name":"setAuthorization","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[{"name":"_link","type":"address"},{"name":"_paymentAmount","type":"uint128"},{"name":"_minimumResponses","type":"uint128"},{"name":"_oracles","type":"address[]"},{"name":"_jobIds","type":"bytes32[]"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"name":"response","type":"int256"},{"indexed":true,"name":"answerId","type":"uint256"},{"indexed":true,"name":"sender","type":"address"}],"name":"ResponseReceived","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"previousOwner","type":"address"}],"name":"OwnershipRenounced","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"previousOwner","type":"address"},{"indexed":true,"name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"id","type":"bytes32"}],"name":"ChainlinkRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"id","type":"bytes32"}],"name":"ChainlinkFulfilled","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"id","type":"bytes32"}],"name":"ChainlinkCancelled","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"current","type":"int256"},{"indexed":true,"name":"roundId","type":"uint256"},{"indexed":false,"name":"timestamp","type":"uint256"}],"name":"AnswerUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"roundId","type":"uint256"},{"indexed":true,"name":"startedBy","type":"address"}],"name":"NewRound","type":"event"}]')

Approximate earliest Ethereum block with price reference response for this oracle.

In [261]:
first_resp_time = contract.functions.getTimestamp(1).call()
current_time = time.time()
latest_block = web3.eth.getBlock('latest')['number']
# one day has 86400 seconds
days_ago = (current_time - first_resp_time) // 86400
# - ~6,000 blocks a day
# - to be safe, we'll add a buffer == 1000
buffer = 1000
blocks_ago = days_ago * (6000 + buffer)
first_block = int(latest_block - blocks_ago)
# assert that the first block we've computed does in fact
# occur before the first price contract response
assert web3.eth.getBlock(first_block )['number'] - first_resp_time < 0

Populate `answers` with all past oracle responses for this price contract. We do this in a roundabout way by querying `ResponseReceived` events.

In [271]:
answers = defaultdict(list)

latest_block = web3.eth.getBlock('latest')['number']
# query in chunks of 1000 blocks
chunk_size = 1000
print("{} chunks".format((latest_block - first_block) // chunk_size))

for chunk in tqdm(range((latest_block - first_block) // chunk_size)):
    
    start_block = first_block + chunk_size * chunk
    
    from_ = start_block
    to_ = start_block + chunk_size
    
    filter_ = contract.events.ResponseReceived().createFilter(
        fromBlock=from_,
        toBlock=to_,
    )
    
    # this shit disconnects often so ...
    # max 100 retries ...
    for i in range(100):
        try:
            filter_ = contract.events.ResponseReceived().createFilter(
                fromBlock=from_,
                toBlock=to_,
            )
            entries = filter_.get_all_entries()
            break
        except ConnectionClosedError:
            if i % 10 == 0:
                print('retrying x{} ...'.format(i))
            time.sleep(5)
            web3 = Web3(Web3.WebsocketProvider(INFURA_PROVIDER))
            contract = web3.eth.contract(ETH_USD_PRICE_CONTRACT_ADDR, abi=CHAINLINK_ABI)
            continue
    else:
        print('Max retries attempted ... still failed')
        break
        
    for res in reversed(entries):
        args = res['args']
        ans_id = args['answerId']
        resp = args['response']
        answers[ans_id].append(resp)


  0%|          | 0/658 [00:00<?, ?it/s]

658 chunks


100%|█████████▉| 656/658 [05:04<00:01,  1.65it/s]

retrying x0 ...
retrying x10 ...
retrying x20 ...
retrying x30 ...
retrying x40 ...
retrying x50 ...
retrying x60 ...
retrying x70 ...
retrying x80 ...
retrying x90 ...


100%|█████████▉| 656/658 [14:48<00:02,  1.35s/it]

Max retries attempted ... still failed


In [273]:
answers

defaultdict(list,
            {1: [16631351520,
              16670000000,
              16614295610,
              16563811450,
              16643715000,
              16592100000,
              16582658130,
              16627988740,
              16582658130,
              16644000000,
              16643436950,
              16638000000,
              16563811450,
              16643331150,
              16582658130,
              16670000000,
              16638000000,
              16563811450,
              16604933280,
              16652082250,
              16592100000],
             2: [17270835810,
              17161291524,
              17282250000,
              17237000000,
              17280000000,
              17238000000,
              17226698380,
              17279136821,
              17161291520,
              17161291524,
              17226698380,
              17237000000,
              17247613160,
              17277853382,
              17276241069,
   

In [275]:
import os
os.path.dirname()

TypeError: dirname() missing 1 required positional argument: 'p'